In [1]:
using Revise

In [2]:
using ForwardDiff

In [3]:
using UnPack

In [4]:
using LinearAlgebra
using ImmersedLayers

In [5]:
using HeatForecastAnyShape

### True data

In [6]:
δ_true = 0.01
zq = [0.5+1.2im]
qq = [1.0]
c1q = [0.5]
c2q = [0.25]
Nq_true = length(zq)

1

### Setting up the observer for the true data

In [7]:
config_true, x_true = get_config_and_state(zq,qq,c1q,c2q)
ϵmeas_data = [1e-5,1e-4,2.5e-4,3.75e-4,5e-4,6.25e-4,8e-4,1e-3,1.5e-3,2e-3,3e-3,4e-3];
x_true

5-element Vector{Float64}:
 0.5
 1.2
 1.0
 0.5
 0.25

### Heater estimation

In [8]:
Nq_estimator = 1
δ = 0.01
config_estimator = HeaterConfig(Nq_estimator)


# ranges to confine the prior mean to
xr = (-2.0,2.0)
yr = (0.01,2) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
c1r = (0,1)
c2r = (0,1)
bounds = create_state_bounds(xr,yr,qr,c1r,c2r,config_estimator);
bounds

5-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (0.01, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

In [9]:
#bounds for numerical domain
xr = (-2.0,2.0)
yr = (-2.0,2.0) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
c1r = (0,1)
c2r = (0,1)
bounds_num = create_state_bounds(xr,yr,qr,c1r,c2r,config_estimator);
bounds_num

5-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (-2.0, 2.0)
 (-2.0, 2.0)
 (0.0, 1.0)
 (0.0, 1.0)

### Set up MCMC parameters

In [10]:
## Parallel tempering ##
propvarX_coarse = 0.01^2 
propvarY_coarse = 0.01^2
propvarQ_coarse = 0.05^2 #0.05^2
propvarC_coarse = 0.01^2

propvarX_fine = 0.005^2 # 0.01^2
propvarY_fine = 0.005^2 # 0.01^2
propvarQ_fine = 0.01^2 # 0.005^2 # 0.01^2
propvarC_fine = 0.005^2

#Ntrial = [10000, 600000]
#propvar = [state_covariance(propvarX_coarse,propvarY_coarse,propvarQ_coarse,propvarR_coarse,config_estimator),
           #state_covariance(propvarX_fine,propvarY_fine,propvarQ_fine,propvarR_fine,config_estimator)];
Ntrial = [10000]
propvar = [state_covariance(propvarX_coarse,propvarY_coarse,propvarQ_coarse,propvarC_coarse,config_estimator)];

### Getting results for a specific case

In [11]:
Δx = 0.01
Ntheta = 500
gridConfig = constructGrids(Δx,bounds_num;Ntheta)

constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 402, 404, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 402, 404, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 402, 404, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 402, 404, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 402, 404, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{402, 404, Float64, true, false}, Edges{Primal, 402, 404, Float64, Vector{Float64}}, Nodes{Dual, 402, 404, Float64, Matrix{Float64}}, Nothing, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Nothing}}(0.01, (-2.0, 2.0), (-2.0, 2.

In [12]:
K = 1
Nsens = 5
ϵmeas = 5e-4
obs_true, ystar, H, Σϵ, Σx = get_truth_data(Nsens,ϵmeas,x_true,config_true,gridConfig)

(TemperatureObservations{5, 5, Vector{ComplexF64}, HeaterConfig{Dict{Any, Any}}}(ComplexF64[-1.0 + 0.0im, -0.5 + 0.0im, 0.0 + 0.0im, 0.5 + 0.0im, 1.0 + 0.0im], HeaterConfig{Dict{Any, Any}}(1, Dict{Any, Any}("heater q" => [3], "heater y" => [2], "heater x" => [1], "heater c1" => [4], "heater c2" => [5]), 0.0)), [-0.8348591168958504, -0.8029232710704499, -0.7710057426082351, -0.7489701713605068, -0.7508582077436514], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [2.5e-7 0.0 … 0.0 0.0; 0.0 2.5e-7 … 0.0 0.0; … ; 0.0 0.0 … 2.5e-7 0.0; 0.0 0.0 … 0.0 2.5e-7], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [13]:
ystar

5-element Vector{Float64}:
 -0.8348591168958504
 -0.8029232710704499
 -0.7710057426082351
 -0.7489701713605068
 -0.7508582077436514

In [14]:
J = ForwardDiff.jacobian(y->analytical_temperature(y,obs_true,gridConfig),x_true)

MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#9#10", Float64}, Float64, 5})
Closest candidates are:
  (::Type{T})(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  (::Type{T})(::T) where T<:Number at boot.jl:772
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number} at char.jl:50
  ...